## Base Output (no reflection)

In [17]:
# Sample inference code for calling LLaMA-3.2-1B-Instruct model.
from openai import OpenAI
import time

openai_api_key = "EMPTY"
openai_api_base = "https://ray-stable-killdeer.ngrok-free.app/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

prompt = "The square quilt block shown is used in a larger quilt. The four small squares in the block are congruent, and the four small vertical rectangles in the block that are not squares are also congruent. The total area of the shaded regions is what fraction of the total area of the block? Express your answer as a common fraction.\n\n[asy]\nfill((-10,0)--(-5,0)--(-5,5)--cycle,gray(0.7));\nfill((-5,5)--(-5,10)--(0,10)--cycle,gray(0.7));\nfill((0,0)--(0,10)--(10,10)--cycle,gray(0.7));\nfill((0,0)--(-10,-10)--(0,-10)--cycle,gray(0.7));\nfill((0,-10)--(5,-10)--(5,-5)--cycle,gray(0.7));\nfill((5,-5)--(5,0)--(10,0)--cycle,gray(0.7));\ndraw((10,10)--(10,-10)--(-10,-10)--(-10,10)--cycle);\ndraw((-10,-10)--(10,10));\ndraw((0,10)--(0,-10));\ndraw((10,0)--(-10,0));\ndraw((-5,0)--(-5,10));\ndraw((5,0)--(5,-10));\n[/asy]"
answer = "3/8"
max_tokens = 1024

time_start = time.time()
chat_response = client.chat.completions.create(
    model="llama-3-1-8b-instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "<|im_start|>system\nPlease reason step by step, and put your final answer within \\boxed{{}}.<|im_end|>\n"
                "<|im_start|>user\n + " + prompt + "<|im_end|>\n"
                "<|im_start|>assistant\n"},
            ],
        },
    ],
    temperature=0.0,
    n=1,
    max_tokens=max_tokens
)

print("Time taken:", time.time() - time_start)
print("\n\nChat response:", chat_response)
print("\n\nChat response text:", chat_response.choices[0].message.content)

Time taken: 7.377032995223999


Chat response: ChatCompletion(id='chat-5197722246d54c90928d661f28acdaa1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To find the fraction of the total area of the block that is shaded, we need to first find the area of the shaded regions and the total area of the block.\n\nLet's denote the side length of the small squares as 's'. \n\nThe area of each small square is s^2.\n\nThe area of each small vertical rectangle is s * (2s) = 2s^2.\n\nThe total area of the block is the sum of the areas of the four small squares and the four small vertical rectangles. \n\nThe area of the four small squares is 4s^2.\n\nThe area of the four small vertical rectangles is 4 * 2s^2 = 8s^2.\n\nThe total area of the block is 4s^2 + 8s^2 = 12s^2.\n\nNow, let's find the area of the shaded regions. \n\nThe shaded region in the top left corner consists of a small square and a small vertical rectangle. \n\nThe area of the sh

## Output With 1 Reflection

In [18]:
# Step 1: Initial Model - Generate reasoning and initial answer
time_start = time.time()
initial_response = client.chat.completions.create(
    model="llama-3-1-8b-instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an assistant tasked with solving a problem step by step. Please:\n"
                "1. Reason step-by-step to solve the problem.\n"
                "2. Provide your initial answer and enclose it in \\boxed{ }."
            )
        },
        {
            "role": "user",
            "content": (
                prompt
            )
        }
    ],
    temperature=0.0,
    n=1,
    max_tokens=max_tokens
)
initial_answer = initial_response.choices[0].message.content
print("Initial Model Response:", initial_answer)

# Step 2: Critic Model - Reflect on the initial answer
critic_response = client.chat.completions.create(
    model="llama-3-1-8b-instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a critic analyzing another AI's reasoning and answer. Please:\n"
                "1. Reflect on the initial reasoning and answer provided.\n"
                "2. Critique the correctness, completeness, and clarity of the response.\n"
                "3. Offer suggestions for improvement."
            )
        },
        {
            "role": "assistant",
            "content": initial_answer
        }
    ],
    temperature=0.0,
    n=1,
    max_tokens=max_tokens
)
critic_feedback = critic_response.choices[0].message.content
print("Critic Model Feedback:", critic_feedback)

# Step 3: Initial Model - Improve the answer using the critique
improved_response = client.chat.completions.create(
    model="llama-3-1-8b-instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an assistant improving your previous response based on feedback. Please:\n"
                "1. Use the feedback from a critic to refine your reasoning.\n"
                "2. Provide a new and improved answer. Enclose it in \\boxed{ }."
            )
        },
        {
            "role": "assistant",
            "content": initial_answer
        },
        {
            "role": "user",
            "content": critic_feedback
        }
    ],
    temperature=0.0,
    n=1,
    max_tokens=max_tokens
)
improved_answer = improved_response.choices[0].message.content
print("Improved Model Response:", improved_answer)

print("Time taken:", time.time() - time_start)


Initial Model Response: To find the fraction of the total area of the block that is shaded, we need to calculate the area of the shaded regions and the total area of the block.

Let's denote the side length of the small squares as $s$ and the width of the small vertical rectangles as $w$. 

The total area of the block is the sum of the areas of the four small squares and the four small vertical rectangles. 

The area of the four small squares is $4s^2$.

The area of the four small vertical rectangles is $4sw$.

So, the total area of the block is $4s^2 + 4sw$.

Now, let's calculate the area of the shaded regions. 

The shaded region in the top left corner consists of a small square and a small vertical rectangle. 

The area of this shaded region is $s^2 + sw$.

There are three more shaded regions, each with the same area.

So, the total area of the shaded regions is $4(s^2 + sw)$.

Now, we can find the fraction of the total area of the block that is shaded by dividing the total area of 